In [1]:
dbutils.fs.put("dbfs:/splunkUF/splunkUF-init.sh" ,"""
ufDlUrl="https://download.splunk.com/products/universalforwarder/releases/8.0.2.1/linux/splunkforwarder-8.0.2.1-f002026bad55-Linux-x86_64.tgz"
ufDlDir="/dbfs/splunkUF"
ufInstallDir="/local_disk0"
TARGETURI="<FQDN of Splunk Deployment Server>:8089"

ufDlFilename=$(basename "${ufDlUrl}")
ufDlPath="${ufDlDir}/${ufDlFilename}"
LCKFILE="${ufDlDir}/.LCKFILE"
SPLUNK_HOME="${ufInstallDir}/splunkforwarder"
SPLUNK_ARGS="--accept-license --answer-yes --no-prompt"

# If Splunk is installed, start splunk and exit
if [ -e ${SPLUNK_HOME} ]; then
  echo "#### ${SPLUNK_HOME} exists! Starting Splunk. ####"
  ${SPLUNK_HOME}/bin/splunk start ${SPLUNK_ARGS}
  exit 0
fi

#sleep if lock file exists
i=0
while [ -e ${LCKFILE} ] && [ $i -lt 30 ]; do
  echo "#### Lock file exists.  Sleeping while ${ufDlFilename} is downloaded to ${ufDlDir} by $(cat ${LCKFILE})####"
  let i=i+1
  sleep 1
done

# If the splunk unervisal forwarder installer does not exist, download it from the splunk website
if ! [ -e ${ufDlPath} ]; then
  if ! [ -e ${ufDlDir} ]; then mkdir -p ${ufDlDir}; fi
  printf $(hostname) > ${LCKFILE}
  echo "#### Downloading ${ufDlFilename} to ${ufDlDir} ####"
  curl -s "${ufDlUrl}" -o ${ufDlPath}
  rm ${LCKFILE}
fi

# exit if the splunk installer file does not exist
if ! [ -e ${ufDlPath} ]; then
  echo "#### ${ufDlPath} not found! Exiting.####"
  exit 1
fi

# Install and configure splunk
echo "#### Installing from ${ufDlPath} ####"
if ! [ -e "${ufInstallDir}" ]; then mkdir "${ufInstallDir}"; fi
tar -xzf ${ufDlPath} -C "${ufInstallDir}"

#Configure deployment client
mkdir -p ${SPLUNK_HOME}/etc/apps/local/default
cat > ${SPLUNK_HOME}/etc/apps/local/default/deploymentclient.conf << EOF
[deployment-client]
clientName = dbr-${HOSTNAME}
[target-broker:deploymentServer]
targetUri = ${TARGETURI}
EOF

#Add DB_CLUSTER_ID & DB_IS_DRIVER as indexed fields/meta data to the events
cat > ${SPLUNK_HOME}/etc/apps/local/default/inputs.conf << EOF
[default]
_meta = DB_CLUSTER_ID::${DB_CLUSTER_ID} DB_IS_DRIVER::${DB_IS_DRIVER}
EOF

${SPLUNK_HOME}/bin/splunk start ${SPLUNK_ARGS}
""", True)